## 카피하며 공부하기 
[데이콘 참고](https://dacon.io/competitions/open/235536/codeshare/2721?page=1&dtype=recent)

\[문화\] 영화 관객수 예측 모델 개발
문화산업 빅데이터를 이용하여 인공지능 모델 개발 

\- 기세현님 코드 분석 & 공부

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("movies_train.csv")
test = pd.read_csv("movies_test.csv")
submission = pd.read_csv('submission.csv')

In [12]:
len(train.distributor.unique().tolist())

159

In [41]:
train.head(30)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11
3,나는 공무원이다,NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9
5,강철대오 : 구국의 철가방,롯데엔터테인먼트,코미디,2012-10-25,113,15세 관람가,육상효,837969.00,2,262,4,233211,9
6,길위에서,백두대간,다큐멘터리,2013-05-23,104,전체 관람가,이창재,NaN,0,32,5,53526,2
7,회사원,쇼박스,액션,2012-10-11,96,청소년 관람불가,임상윤,739522.00,3,342,2,1110523,11
8,"1789, 바스티유의 연인들",유니버설픽쳐스인터내셔널코리아,뮤지컬,2014-09-18,129,전체 관람가,정성복,NaN,0,3,5,4778,1
9,청춘그루브,두타연,드라마,2012-03-15,94,15세 관람가,변성현,NaN,0,138,3,868,8


In [6]:
submission.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


In [7]:
# 배급사 전처리
train['distributor'] = train.distributor.str.replace('(주)','')
test['distributor'] = test.distributor.str.replace('(주)','')
train['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]','',x) for x in train.distributor]
test['distributor'] = [re.sub('[^0-9a-zA-Z가-힣]','',x) for x in test.distributor]

In [13]:
def get_dis(x):
  if 'CJ' in x or 'CGV' in x:
    return 'CJ'
  elif '쇼박스' in x:
    return '쇼박스'
  elif 'SK' in x:
    return 'SK'
  elif '리틀빅픽' in x:
    return '리틀빅픽처스'
  elif '스폰지' in x:
    return '스폰지'
  elif '싸이더스' in x:
    return '싸이더스'
  elif '에이원' in x:
    return '에이원'
  elif '마인스' in x:
    return '마인스'
  elif '마운틴픽' in x:
    return '마운틴픽처스'
  elif '디씨드' in x:
    return '디씨드'
  elif '드림팩트' in x:
    return '드림팩트'
  elif '메가박스' in x:
    return '메가박스'
  elif '마운틴' in x:
    return '마운틴'
  else:
    return x

In [14]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

In [16]:
len(train.distributor.unique().tolist())

147

In [30]:
train.groupby('genre').mean().box_off_num.sort_values()

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [31]:
train.groupby('genre').box_off_num.mean().sort_values()


genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [32]:
train['genre_rank'] = train.genre.map({'뮤지컬':1,'다큐멘터리':2,'서스펜스':3,'애니메이션':4,'멜로/로맨스':5,'미스터리':6,'공포':7,'드라마':8,'코미디':9,'SF':10,'액션':11,'느와르':12})
test['genre_rank'] = test.genre.map({'뮤지컬':1,'다큐멘터리':2,'서스펜스':3,'애니메이션':4,'멜로/로맨스':5,'미스터리':6,'공포':7,'드라마':8,'코미디':9,'SF':10,'액션':11,'느와르':12})

In [33]:
train.genre_rank

0      11
1      12
2      11
3       9
4       9
       ..
595     8
596     8
597     7
598    12
599    11
Name: genre_rank, Length: 600, dtype: int64

In [34]:
#배급사별 영화 관색수 중위값 기준으로 배급사 랭크 인코딩
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name='num_rank').sort_values(by='num_rank')

In [39]:
type(train.groupby('distributor').box_off_num.median()) ## 시리즈 형태에서 사용할 수 있는 reset_index(name='')을 적용하여 데이터 프레임으로 확장

pandas.core.series.Series

In [48]:
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,2.0
15,고구마공작소,8.0
52,사람과사람들,42.0
97,위드시네마,46.0
19,나우콘텐츠,54.0
...,...,...
113,전망좋은영화사,1214237.0
105,이십세기폭스코리아,1422844.0
56,쇼박스,2138560.0
84,영구아트무비,2541603.0


In [49]:
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])] ## len(tr_nm_rank)과 같은 의미

In [50]:
tr_nm_rank ######## 배급사에 순서를 매김 

,distributor,num_rank
110,인피니티엔터테인먼트,1
15,고구마공작소,2
52,사람과사람들,3
97,위드시네마,4
19,나우콘텐츠,5
...,...,...
113,전망좋은영화사,143
105,이십세기폭스코리아,144
56,쇼박스,145
84,영구아트무비,146


In [54]:
train = pd.merge(train, tr_nm_rank, how = 'left')
test = pd.merge(test, tr_nm_rank,how = 'left')
test.fillna(0,inplace = True)  #### test 데이터 셋에 기존 train 데이터셋에 없던 배급사가 나올 경우 결측치가 됨 이것을 0으로 대체

In [55]:
train

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11,134
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,145
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,145
3,나는 공무원이다,NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,140
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,NEW,드라마,2014-08-13,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,140
596,파파로티,쇼박스,드라마,2013-03-14,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,145
597,살인의 강,마운틴픽처스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475,7,46
598,악의 연대기,CJ,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525,12,141


In [56]:
test

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_rank,num_rank
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,12,132.0
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,5,145.0
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,8,141.0
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2,11,145.0
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1,7,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2.590000e+03,1,4,4,8,112.0
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,0.000000e+00,0,18,2,2,69.0
240,어떤살인,컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,0.000000e+00,0,224,4,12,0.0
241,말하지 못한 비밀,마운틴픽처스,드라마,2015-10-22,102,청소년 관람불가,송동윤,5.069900e+04,1,68,7,8,46.0


In [60]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#!pip install catboost #코랩에 catboost가 없음
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
#!pip install ngboost # 코랩에 ngboost도 없음
from ngboost import NGBRegressor

     |████████████████████████████████| 348 kB 8.7 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=b35a075f9f8176a1d894ec4ce48250d7619620f3309978b4dabee5f4aa1b16cb
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma
